# PROYECTO OPEN DATA II

### IMPLEMENTACIÓN DEL ALGORÍTMO DE REGRESIÓN LINEAL

### Introducción

Para la segunda parte de esta asignatura nos hemos sumergido en el mundo de programación con pyspark, utilizado en la dinámica de programación en distribuido. A través de pyspark hemos trabajado con un algoritmo de machine learning, asi pues pudiendo trabajar con las bases de uno de los fenómenos más populares en el mercado ahora mismo.

El problema que presenta nuestro dataset, Graduate Admissions, obtenido de la página web kaggle.com, es predecir la posibilidad que tiene un alumno específico, basándose en sus calificaciones y otras carácterísticas que listaremos más adelante, en ser admitido para cursar un máster en una universidad en particular. Para ello, se nos proporciona al inicio una variedad de parámetros que son considerados importantes a la hora de realizar la apliación para los programas de máster. 
Los parámetros son los siguientes:

1. GRE Scores ( de 0 a 340 )
2. TOEFL Scores ( de 0 a120 )
3. Valoración de la universidad (de 0 a 5 )
4. Declaración de propósito y carta de recomendación (de 0 a 5)
5. GPA Scores (de 0 a 10)
6. Experiencia en investigación (0 o 1)
7. Probabilidad de ser admitido (entre 0 y 1)

Para poder realizar el objetivo del dataset, es decir, predecir la posibilidad de poder entrar, hemos optado por la implementación de un algoritmo de regresión lineal. 
La elección de este en concreto se basa en que su propósito es establecer un modelo para la relación entre características y una variable objetivo, por lo que se ajusta muy correctamente a nuestras necesidades. Para el dataset con el que trabajamos nosotras, la primera consiste de una serie de calificaciones optenidas por cierto estudiante y la segunda será la probabilidad de entrada de este estudiante en un máster en concreto.

Aplicamos nuestro algoritmo, y lo utilizamos para calcular algunos valores que nos ayudan a medir la eficacia de este. A continuación manejamos una mejora del modelo en cuestión, empleando técnicas de hyper-tuning de los parámetros y grid search, y acabando con una extracción de características y PCA.

**Preparación del entorno de trabajo**

Montamos el entorno de programcación con pyspark, importandonos los módulos necesarios.
En este caso trabajamos con SparkContext, SparkSession y SQLContext.

In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext
sc=SparkContext(master="local[3]")
print(sc)
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

A continuación nos descargamos el dataset y realizamos la limpieza necesaria.

In [ ]:
df = sqlContext.read.format("com.databricks.spark.csv").options(header='true',inferschema='true').load("Admission_Predict.csv")
display(df)

Solo hemos considerado necesario cambiar el nombre de una columna, ya que tenía un punto que en algunos métodos daba problemas.

In [ ]:
df = df.withColumnRenamed("Serial No.", "Serial No")

Comprobamos el dataset.

In [ ]:
df.show(5)

In [ ]:
df.printSchema()

**Alogritmo de Regresión Lineal**

En primer lugar, convertimos los datos a dense vector: creamos features y label, las etiquetas con las que suelen trabajar los algoritmos machine learning.

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [ ]:
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

In [ ]:
transformed= transData(df)
transformed.show(5)

In [ ]:
from pyspark.ml.feature import VectorIndexer

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures").fit(transformed)
data = featureIndexer.transform(transformed)

In [ ]:
data.show(5,True)

Separamos los datos en datos de entrenamiento (training) y de testeo (test) (60% para training y 40% para testing)

In [ ]:
(trainingData, testData) = transformed.randomSplit([0.6, 0.4])

In [ ]:
trainingData.show(5)
testData.show(5)

Definimos algoritmo de Regresion Lineal. Tiene tres parámetros, a los que para empezar ponemos valores por defecto.

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=2, regParam=0.5, elasticNetParam=0.5)

Montamos arquitectura Pipeline (segmentación de instrucciones): Permite implementar el paralelismo a nivel de instrucción en un único procesador.

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[featureIndexer, lr])
model = pipeline.fit(trainingData)

Obtenemos un resumen de nuestro modelo. Sacamos los resultados de los siguientes:
* **Error cuadrático medio**: un estimador que mide el promedio de los errores al cuadrado, es decir, la diferencia entre el estimador y lo que se estima
* **RMSE**: mide las diferencias entre los valores predichos por un modelo o un estimador y los valores observados.
* **R cuadrado**: (coeficiente de determinación) para predecir futuros resultados/probar una hipótesis (como de bien se pueden predecir futuros resultados). Se puede interpretar con como de cerca están los datos a la linea de regresión

In [ ]:
lrModel = model.stages[-1]
trainingSummary = lrModel.summary

In [ ]:
def modelsummary(model):
    import numpy as np
    Summary=model.summary
    
    print ("##",'-----------------------------------------------------')
    print ("##","Error Cuadrático Medio: % .6f" \
           % Summary.meanSquaredError, ", RMSE: % .6f" \
           % Summary.rootMeanSquaredError )
    print ("##","R cuadrado: %f" % Summary.r2, ", \
    Total iteraciones: %i"% Summary.totalIterations)
    print ("##",'-----------------------------------------------------')

In [ ]:
modelsummary(model.stages[-1])

Procedemos a realizar predicciones con el test data:

In [ ]:
predictions = model.transform(testData)

In [ ]:
predictions.select("features","label","prediction").show(5)

**Evaluación de resultados**

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Raiz del error cuadrático medio (RMSE) de los datos de prueba = %g" % rmse)

In [ ]:
y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

In [ ]:
import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred) 
print('valor r2_: {0}'.format(r2_score))

**Mejora del modelo: Hyper-tunning**

El hyper-tunning trata de encontrar la mejor opción de combinación de valores de los parámetros de entrada de nuestro modelo. De tal forma podemos entrenar nuestro modelo previo para encontrar su rendimiento más óptimo.

**Grid search** es un algoritmo que itera a través de la lista de valores de los parámetros y estima los modelos de manera independiente y escoje la mejor opción.

En primer lugar, especificamos a nuestro modelo la lista de parámetros sobre la que vamos a iterar.

In [ ]:
import pyspark.ml.tuning as tune
import pyspark.ml.classification as cl
import pyspark.ml.evaluation as ev

linear = LinearRegression(labelCol='label',featuresCol = 'indexedFeatures')
grid = tune.ParamGridBuilder().addGrid(linear.maxIter, [2, 10, 30]).addGrid(linear.regParam, [0.01, 0.07, 0.5]).addGrid(linear.elasticNetParam, [0.5, 0.4, 0.8]).build()

# BinaryClassificationEvaluator lo utilizamos para comparar los modelos (a través de la comparación de su rendimiento)
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')

In [ ]:
cv = tune.CrossValidator(estimator=linear, estimatorParamMaps=grid, evaluator=evaluator)

In [ ]:
pipeline2 = Pipeline(stages=[featureIndexer])
model2 = pipeline2.fit(trainingData)

cvModel nos devuelve el mejor modelo estimado.

In [ ]:
cvModel = cv.fit(model2.transform(trainingData))

Sacamos el area bajo la curva ROC y PR

In [ ]:
data_train = model2.transform(testData)
results = cvModel.transform(data_train)

print(evaluator.evaluate(results, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(results, {evaluator.metricName: 'areaUnderPR'}))

**Evaluación de los mejores parámetros para nuestro modelo**

In [ ]:
results = [
    (
        [
            {key.name: paramValue} 
            for key, paramValue 
            in zip(
                params.keys(), 
                params.values())
        ], metric
    ) 
    for params, metric 
    in zip(
        cvModel.getEstimatorParamMaps(), 
        cvModel.avgMetrics
    )
]

sorted(results, key=lambda el: el[1], reverse=True)[0]

In [ ]:
print ('Best Param (MaxIter): ', cvModel.bestModel._java_obj.getMaxIter())

In [ ]:
print ('Best Param (RegParam): ', cvModel.bestModel._java_obj.getRegParam())

In [ ]:
print ('Best Param (ElasticNetParam): ', cvModel.bestModel._java_obj.getElasticNetParam())

Una vez obtenidos los mejores parámetros para nuestro modelo, lo entrenamos con estos parámetros para obtener la mejor predicción en base a nuestros datos de entrada.

### Conclusiones

En primer lugar, queremos remarcar el conocimiento obtenido en cuanto a los sistemas distribuídos, tanto su funcionalidad como la cantidad de diferentes opciones que existen hoy en día para poder hacer uso de ellos, ya que permiten utilizar grandes cantidades de datos sin tener que contar físicamente con un equipo extremadamente moderno y preparado para soportar estas grandes cantidades de datos. Ya que, aunque en nuestro caso específico, no teníamos muchos problemas con el tamaño del dataset, sabemos que en el futuro trabajaremos con el doble y el triple de datos, por lo que nos será de gran ayuda haber conocido de los sistemas distribuídos. 

En segundo lugar, nos gustaría mencionar que haber podido trabajar con pyspark nos abre un nuevo campo de trabajo aún por desarrollar, pero que tiene muchas ventajas en comparación con python que es con lo que trabajamos el anterior curso. Una ventaja que hemos percibido es que es muy veloz a la hora de computacionar, ya que con la cantidad de datos que tenemos, se obtiene una rápida respuesta, y se podría haber hecho mucho más complicado trabajar con un gran volumen de datos de la otra manera. Además, permite usar librerías como MLLib que proporcionan una gran variedad de algoritmos machine learning como regresión logítica, lineal, random forest, etc. Estos algoritmos son perfectos para ampliar nuestro conocimento en el tema y verlo de manera práctica. Aunque tiene como desventaja que es un sistema muy nuevo y existe muy poca documentación sobre él, sabemos que en un futuro cercano esto cambiará y las ventajas serán totales.

Por otro lado, el conocimiento sobre los algoritmos de machine learning es otro tema que nos ha marcado durante el transcurso de la asignatura, ya que es un tema nuevo para nosotras con el que nunca habíamos trabajado y el aprender, tanto de manera teórica con las presentaciones primeras como de manera práctica en nuestro proyecto, nos ha enriquecido mucho. Además, gracias a estos, hemos aprendido a mejorar los modelos de manera que el valor que aportan los datos, tanto a nosotras de manera educacional como a las empresas para obtener beneficio de estos, sea mayor y tenga una mejor precisión. 

En cuanto a los resultados que hemos obtenido para la predicción de entrada a una universidad, hemos de remarcar que, la aplicación de las mejoras ha hecho que redujésemos el error cuadrático medio, el cual cuanto menor fuese, mejor sería la predicción. Obtuvimos sin aplicar ninguna mejora un ECM de 0.020416. Tras aplicar hyper-tunning y obteniendo los mejores parámetros para la regresión lineal un ECM de 0.003526. Por último, aplicando Chi-Cuadrado y PCA un ECM de 0.003518. No hay apenas diferencia en los últimos datos, pero pensamos que esto es porque no tenemos una gran cantidad de parámetros por lo que esto no afectará en gran medida a los cálculos, pero en futuros datasets podría ser de gran ayuda.

Por último, creemos que esta asignatura nos ha proporcionado una visión más amplia del mundo profesional en el que vamos a convivir en unos años, ya que pyspark, como he mencionado antes, será uno de los principales métodos para trabajar en el análisis de datos y haber trabajado con esto, nos muestra como podría ser nuestro futuro profesional y nos aporta una gran ventaja competitiva frente a otros que no lo hayan usado.
